### Fraud Transactions Classification Performance Analysis using Deep Learning Approach
 Fraudulent transactions have increased as a result of the huge number of transactions committed every day. Data storage systems and the digitalization of world business are targets for criminals because they store sensitive information. This information was used to commit illegal transactions. 

We propose a solution to this problem in this paper, which can be described as follows:
: To overcome
the imbalanced dataset issue, an autoencoder combined with the support
vector machine model (ASVM) was applied. For the classification phase,
we are using an attention-long short-term memory neural network as a
weak learner for the gradient-boosting algorithm (GB ALSTM), compared with the following techniques: artificial neural networks (ANN),
convolutional neural networks (CNN), long short-term memory neural
networks (LSTM), attention-long short-term memory neural networks
(ALSTM), and bidirectional long short-term memory neural networks
(BLSTM). Several experiments have been applied to a real-world dataset,
showing promising results in detecting abnormal transactions and highlighting the dominance of our suggested solution against the competing
models.

In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Attention,Bidirectional,Dropout,Layer,Embedding,Dropout, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, KFold, StratifiedKFold, train_test_split
from sklearn.metrics import plot_roc_curve, precision_score,  precision_recall_curve,plot_precision_recall_curve,average_precision_score, recall_score, f1_score, roc_auc_score, roc_curve, accuracy_score, precision_recall_curve, confusion_matrix
from sklearn.svm import SVC 
from imblearn.under_sampling import RandomUnderSampler 
from scipy.optimize import minimize_scalar

In [79]:
def Evaluation_metrics(y_test,y_pred):
    Accuracy=accuracy_score(y_test,y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    Sensitivity=tp / (tp+fn)
    Specificity=tn / (tn+fp)
    Precision=tp / (tp+fp) 
    f1_score=(2*(tp / (tp+fp))*(tp / (tp+fn)))/ ( (tp / (tp+fp))+(tp / (tp+fn))) 
    Confusion_matrics=confusion_matrix(y_test,y_pred)
    return Accuracy,Sensitivity,Specificity,Precision,f1_score,Confusion_matrics

In [8]:
#Import the dataset 
df=pd.read_csv('df_1.csv') 

In [9]:
X=df.drop('Class',axis=1)
y=df.Class

In [10]:
# Split the data into training and testing data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.3,random_state=42)

In [11]:
X_train_fraud_samples=X_train[y_train==1]
X_train_legitimate_samples=X_train[y_train==0]

In [12]:
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train, y_train)
model=SVC()
model.fit(X_res,y_res)

SVC()

In [15]:
X_train_fraud_samples.shape

(356, 30)

####  Oversampling the training set using a Support vector machine algorithm with an autoencoder architecture

#Encoder 30-->23-->19-->17-->8    
#Decoder 8-->17-->19-->23-->30

In [17]:
#Using Autoencoder+SVM to balance the dataset
while X_train_fraud_samples.shape[0]<=X_train_legitimate_samples.shape[0] :
    encoder = Sequential()
    encoder.add(Dense(units=23,activation='relu',input_shape=[30]))   
    encoder.add(Dropout(0.1))
    encoder.add(Dense(units=19,activation='relu'))
    encoder.add(Dropout(0.2))
    encoder.add(Dense(units=17,activation='relu'))
    encoder.add(Dense(units=8,activation='relu'))
#Encoder 30-->23-->19-->17-->8    
#Decoder 8-->17-->19-->23-->30
    decoder = Sequential()
    decoder.add(Dense(units=17,activation='relu',input_shape=[8]))
    encoder.add(Dropout(0.2))
    decoder.add(Dense(units=19,activation='relu'))
    encoder.add(Dropout(0.1))
    decoder.add(Dense(units=23,activation='relu'))
    decoder.add(Dense(units=30,activation='relu'))
    autoencoder = Sequential([encoder,decoder])
    autoencoder.compile(loss='binary_crossentropy' ,optimizer='adam')
    autoencoder.fit(X_res,X_res,epochs=100,batch_size=40,shuffle=True) 
    X_train_fraud_samples_generated=autoencoder.predict(X_train_fraud_samples)
    X_train_fraud_samples_generated=pd.DataFrame(X_train_fraud_samples_generated)
    y_pred=model.predict(X_train_fraud_samples_generated)
    fraud_samples=X_train_fraud_samples_generated[y_pred==1]
    fraud_samples.columns=X_train_fraud_samples.columns
    X_train_fraud_samples=pd.concat([X_train_fraud_samples,fraud_samples],axis=0)
    print('X_train_fraud_samples>>>',X_train_fraud_samples.shape) 
    
    

Epoch 1/100
18/18 [==============================] - 1s 3ms/step - loss: -7.1291
Epoch 2/100
18/18 [==============================] - 0s 3ms/step - loss: -8.6618
Epoch 3/100
18/18 [==============================] - 0s 3ms/step - loss: -9.9461
Epoch 4/100
18/18 [==============================] - 0s 3ms/step - loss: -11.0544
Epoch 5/100
18/18 [==============================] - 0s 3ms/step - loss: -12.0613
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: -12.1964
Epoch 7/100
18/18 [==============================] - 0s 2ms/step - loss: -12.5519
Epoch 8/100
18/18 [==============================] - 0s 2ms/step - loss: -12.9269
Epoch 9/100
18/18 [==============================] - 0s 2ms/step - loss: -13.1844
Epoch 10/100
18/18 [==============================] - 0s 3ms/step - loss: -13.4358
Epoch 11/100
18/18 [==============================] - 0s 2ms/step - loss: -13.6441
Epoch 12/100
18/18 [==============================] - 0s 2ms/step - loss: -14.0412
Epoch 13/100
18/

C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


X_train_fraud_samples>>> (80127, 30)
Epoch 1/100
18/18 [==============================] - 1s 3ms/step - loss: -4.4167
Epoch 2/100
18/18 [==============================] - 0s 3ms/step - loss: -7.5743
Epoch 3/100
18/18 [==============================] - 0s 3ms/step - loss: -9.1680
Epoch 4/100
18/18 [==============================] - 0s 5ms/step - loss: -10.2027
Epoch 5/100
18/18 [==============================] - 0s 2ms/step - loss: -11.5002
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: -12.1036
Epoch 7/100
18/18 [==============================] - 0s 3ms/step - loss: -12.6706
Epoch 8/100
18/18 [==============================] - 0s 3ms/step - loss: -12.9877
Epoch 9/100
18/18 [==============================] - 0s 2ms/step - loss: -12.9306
Epoch 10/100
18/18 [==============================] - 0s 2ms/step - loss: -13.2600
Epoch 11/100
18/18 [==============================] - 0s 3ms/step - loss: -13.3229
Epoch 12/100
18/18 [==============================] - 0s 2ms/s

C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


X_train_fraud_samples>>> (89978, 30)
Epoch 1/100
18/18 [==============================] - 1s 3ms/step - loss: -4.7053
Epoch 2/100
18/18 [==============================] - 0s 3ms/step - loss: -7.8435
Epoch 3/100
18/18 [==============================] - 0s 2ms/step - loss: -10.3134
Epoch 4/100
18/18 [==============================] - 0s 2ms/step - loss: -12.1157
Epoch 5/100
18/18 [==============================] - 0s 3ms/step - loss: -12.6420
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: -12.9735
Epoch 7/100
18/18 [==============================] - 0s 2ms/step - loss: -13.1418
Epoch 8/100
18/18 [==============================] - 0s 3ms/step - loss: -13.3626
Epoch 9/100
18/18 [==============================] - 0s 2ms/step - loss: -13.6027
Epoch 10/100
18/18 [==============================] - 0s 2ms/step - loss: -13.7603
Epoch 11/100
18/18 [==============================] - 0s 2ms/step - loss: -13.9593
Epoch 12/100
18/18 [==============================] - 0s 3ms/

...

X_train_fraud_samples>>> (196208, 30)
Epoch 1/100
18/18 [==============================] - 1s 2ms/step - loss: -8.1475
Epoch 2/100
18/18 [==============================] - 0s 2ms/step - loss: -9.7049
Epoch 3/100
18/18 [==============================] - 0s 2ms/step - loss: -10.8070
Epoch 4/100
18/18 [==============================] - 0s 2ms/step - loss: -11.9410
Epoch 5/100
18/18 [==============================] - 0s 2ms/step - loss: -12.7607
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: -13.5575
Epoch 7/100
18/18 [==============================] - 0s 2ms/step - loss: -13.9832
Epoch 8/100
18/18 [==============================] - 0s 2ms/step - loss: -14.2320
Epoch 9/100
18/18 [==============================] - 0s 2ms/step - loss: -14.5155
Epoch 10/100
18/18 [==============================] - 0s 2ms/step - loss: -14.7442
Epoch 11/100
18/18 [==============================] - 0s 2ms/step - loss: -15.1005
Epoch 12/100
18/18 [==============================] - 0s 2ms

C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


X_train_fraud_samples>>> (196318, 30)
Epoch 1/100
18/18 [==============================] - 1s 2ms/step - loss: -8.2365
Epoch 2/100
18/18 [==============================] - 0s 2ms/step - loss: -10.2631
Epoch 3/100
18/18 [==============================] - 0s 2ms/step - loss: -11.4752
Epoch 4/100
18/18 [==============================] - 0s 2ms/step - loss: -12.1409
Epoch 5/100
18/18 [==============================] - 0s 2ms/step - loss: -12.7761
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: -13.1612
Epoch 7/100
18/18 [==============================] - 0s 2ms/step - loss: -13.3367
Epoch 8/100
18/18 [==============================] - 0s 2ms/step - loss: -13.3728
Epoch 9/100
18/18 [==============================] - 0s 2ms/step - loss: -13.7202
Epoch 10/100
18/18 [==============================] - 0s 2ms/step - loss: -13.8489
Epoch 11/100
18/18 [==============================] - 0s 2ms/step - loss: -13.8949
Epoch 12/100
18/18 [==============================] - 0s 2m

C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


X_train_fraud_samples>>> (263623, 30)


### Combine the fraud transaction samples generated with the legitimate transaction after that we gets the new oversamples training dataset(X_train,y_train)

In [18]:
X_train_fraud_samples['Class']=1
X_train_legitimate_samples['Class']=0
df_train=pd.concat([X_train_fraud_samples,X_train_legitimate_samples],axis=0)
shuffled = df_train.sample(frac=1).reset_index()
X_train=shuffled.drop(["Class","index"],axis=1)
y_train=shuffled.Class

C:\Users\HP\AppData\Local\Temp/ipykernel_10528/1830853257.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_legitimate_samples['Class']=0


In [102]:
X_train=shuffled.drop(["Class","index"],axis=1)
y_train=shuffled.Class

In [103]:
X_train.shape,y_train.value_counts()

((462631, 30),
 1    263623
 0    199008
 Name: Class, dtype: int64)

In [117]:
def fit_gradient_boosting(X, y, n_estimators=100):
     # Initialize
    n_samples,t, n_features = X.shape
    n_estimators = 5
    estimators = []
    F = np.full((n_samples, ), 0.0)  # Predictions of each training example using the ensemble
    
    for t in range(n_estimators):
        # Fit a weak learner to the residuals, which are computed as gradient(Loss(y, F))
        residuals = y - F
        h=get_model()
        h.fit(X,residuals,epochs=100,batch_size=3000,shuffle=True)
        #h = DecisionTreeRegressor(max_depth=1)
        #h.fit(X, residuals)
    
        # Compute a step length that produces the best improvement in the loss
        hreg = h.predict(X,verbose=0).flatten()
        loss = lambda a: np.linalg.norm(y - (F + a * hreg))**2
        print(loss)
        step = minimize_scalar(loss, method='golden')
        print('step')
        a = step.x

        # Update the ensemble predictions
        F += a * hreg

        # Update the ensemble
        estimators.append((a, h))
    
    return estimators 
def predict_gradient_boosting(X, estimators):
    pred = np.zeros((X.shape[0], ))

    for a, h in estimators:
        pred += a * h.predict(X).flatten()

    y = np.sign(pred)

    return y

In [118]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()
def get_model():  
    inputs = Input(shape=(1,30))
    x, x_h, x_c = LSTM(10, return_sequences=True, return_state=True)(inputs)
    x = Attention(30)([x, x_h, x_c])
    x = Dropout(0.2)(x)
    x = Dense(8, activation="relu")(x)
    output = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Precision()])
    return model

## Attention long short-term memory as a weak learner for the Gradient Boosting Algorithm

In [109]:
GB=GradientBoostingClassifier(7,0.001)

In [105]:
X_train_.shape ,X_train.shape

((462631, 1, 31), (462631, 30))

In [119]:
X_train_=X_train.values.reshape(X_train.shape[0],1,X_train.shape[1])
estimators = fit_gradient_boosting(X_train_, y_train) 

Epoch 1/100
155/155 [==============================] - 94s 588ms/step - loss: 0.4914 - precision_13: 0.6666
Epoch 2/100
155/155 [==============================] - 91s 586ms/step - loss: 0.1375 - precision_13: 0.9963
Epoch 3/100
155/155 [==============================] - 94s 604ms/step - loss: 0.0412 - precision_13: 0.9995
Epoch 4/100
155/155 [==============================] - 91s 587ms/step - loss: 0.0151 - precision_13: 0.9996
Epoch 5/100
155/155 [==============================] - 92s 595ms/step - loss: 0.0079 - precision_13: 0.9998
Epoch 6/100
155/155 [==============================] - 91s 589ms/step - loss: 0.0054 - precision_13: 0.9998
Epoch 7/100
155/155 [==============================] - 94s 605ms/step - loss: 0.0042 - precision_13: 0.9998
Epoch 8/100
155/155 [==============================] - 92s 590ms/step - loss: 0.0034 - precision_13: 0.9998
Epoch 9/100
155/155 [==============================] - 91s 584ms/step - loss: 0.0030 - precision_13: 0.9998
Epoch 10/100
155/155 [======

KeyboardInterrupt: 

## Define the LSTM, BLSTM, ANN, CNN models

In [54]:
# Define the LSTM model
lstm = Sequential()
lstm.add(LSTM(10, input_shape=(30, 1)))
lstm.add(Dense(1))
lstm.compile(loss='mean_squared_error', optimizer='adam')

# Define the Bidercional LSTM model
Bider_lstm = Sequential()
Bider_lstm.add(Bidirectional(LSTM(10,input_shape=(1, 30))))
Bider_lstm.add(Dense(1,'sigmoid'))
Bider_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Precision()])

# Create a sequential model
ann = Sequential()
ann.add(Dense(23, activation='relu', input_shape=(30,)))
ann.add(Dense(19, activation='relu'))
ann.add(Dense(8, activation='relu'))
ann.add(Dense(1, activation='softmax'))
ann.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Precision()])

cnn = Sequential()
cnn.add(keras.layers.Conv1D(23, 2, activation = "relu",input_shape=(X_train.shape[1], X_train.shape[-1])))
cnn.add(keras.layers.Dropout(0.1))
cnn.add(keras.layers.BatchNormalization())
cnn.add(keras.layers.Conv1D(46, 2, activation = "relu"))
cnn.add(keras.layers.Dropout(0.2))          # prevents over-fitting (randomly remove some neurons)
cnn.add(keras.layers.Flatten())
cnn.add(keras.layers.Dropout(0.2))
cnn.add(keras.layers.Dense(8, activation = "relu"))
cnn.add(keras.layers.Dropout(0.1))
cnn.add(keras.layers.Dense(1, activation = "sigmoid"))
cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Precision()])

In [77]:
classifiers={"lstm":lstm,"lstmb":Bider_lstm,"att_lstm":get_model(),"cnn":cnn,  "ann":ann}   

In [ ]:
Accuracy_values=[]
Sensitivity_values=[]
Specificity_values=[]
Precision_values=[]
f1_score_values=[]
diff_time=[]
classifier_value=[]
Confusion_matrics_values=[]

for key, classifier in classifiers.items():
    start_time=time.time()
    if key!="ann" and key!="cnn":
        print(key)
        X_train_=X_train.to_numpy().reshape(X_train.shape[0],1,X_train.shape[1])
        X_test_=X_test.to_numpy().reshape(X_test.shape[0],1,X_test.shape[1])
        classifier.fit(X_train_,y_train,epochs=100,batch_size=200,shuffle=True) 
        y_pred= classifier.predict_classes(X_test_).ravel()
    if key=="cnn":
        print("cnn")
        X_train_ = X_train.to_numpy().reshape(X_train.shape[0],X_train.shape[1],1)
        X_test_ = X_test.to_numpy().reshape(X_test.shape[0],X_test.shape[1],1)
        classifier.fit(X_train_, y_train, epochs = 100,batch_size=50, verbose=1)
        y_pred= classifier.predict_classes(X_test_)
    if key=="ann":
          classifier.fit(X_train, y_train, epochs = 100,batch_size=50, verbose=1)
          y_pred= classifier.predict_classes(X_test)
    end_time=time.time()
    Accuracy,Sensitivity,Specificity,Precision,f1_score,Confusion_matrics=Evaluation_metrics(y_test,y_pred)
    Accuracy_values.append(Accuracy)
    Sensitivity_values.append(Sensitivity)
    Specificity_values.append(Specificity)
    Precision_values.append(Precision)
    f1_score_values.append(f1_score)
    Confusion_matrics_values.append(Confusion_matrics)
    classifier_value.append(key)
    time_value=end_time-start_time
    diff_time.append(time_value)
    print('End  ')
Data=pd.DataFrame({"classifiers":classifier_value,"Acc":Accuracy_values,"Sens":Sensitivity_values,
                   "Sep":Specificity_values,"Pre":Precision_values,
                   "f1_score":f1_score_values,"Time":diff_time,"ConfusionMatrix":Confusion_matrics_values }) 
Data.to_excel("Creditcard_resutsv2.xlsx",sheet_name='Eropean_credit_card_dataset') 
    